In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


It will be first time that I use ML models, I hope you will like it :)

Loading data and looking details

In [ ]:
df=pd.read_csv("../input/brasilian-houses-to-rent/houses_to_rent.csv")

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
list=[]
for i in df["total"]:
    x=i[2:]
    x=x.replace(",","")
    list.append(x)
df=df.drop("total",axis=1)
df["total rent"]=list   
df["total rent"]=df["total rent"].astype(int)
df['animal'] = df['animal'].map({'acept': 1, 'not acept': 0})
df['furniture'] = df['furniture'].map({'furnished': 1, 'not furnished': 0})
df.head()

In [ ]:
df=df.drop(["Unnamed: 0","hoa","rent amount","property tax","fire insurance"],axis=1)
df["floor"]=df["floor"].replace("-","0")
df["floor"]=df["floor"].astype(int)
df.head()

Now we can look the correlations

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),annot=True)

In [ ]:
column_list=["city","area","rooms","bathroom","parking spaces","floor","animal","furniture"]

In [ ]:
for i in column_list:
    plt.subplots()
    plt.scatter(df[i]/df[i].max(),df["total rent"]/df["total rent"].max())
    plt.xlabel(i)
    plt.ylabel("total rent")

    

Lets predict

In [ ]:
y=df["total rent"]
x=df[["city","area","rooms","bathroom","parking spaces","floor","animal","furniture"]]

In [ ]:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
lr=LinearRegression()
for a in np.arange(1,6):
    pl=PolynomialFeatures(degree=a)
    x_train_polly=pl.fit_transform(x_train)
    lr.fit(x_train_polly,y_train)
    x_test_polly=pl.fit_transform(x_test)
    y_head=lr.predict(x_test_polly)
    from sklearn.metrics import r2_score
    print("for {}. degree r^2 score is".format(a),r2_score(y_test,y_head))
    plt.subplots()
    plt.figure(figsize=(15,10))
    ax1=sns.distplot(y_test,hist=False,color="r",label="Actual value")
    sns.distplot(y_head,hist=False,color="g",label="Predicted Value")
    plt.title("{}.degree".format(a))
    

Linear or polynomial regressions are not efficient for this dataset as we can see.So lets use random forest.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor(n_estimators=100,random_state=42)
rf.fit(x_train,y_train)
y_head=rf.predict(x_test)
print("r^2 score is ",r2_score(y_test,y_head))
plt.figure(figsize=(15,10))
ax1=sns.distplot(y_test,hist=False,color="r",label="Actual value")
sns.distplot(y_head,hist=False,color="g",label="Predicted Value")

It is not bad I think we can use this model.

In [ ]:
value_list=[]
for i in ["city","area","rooms","bathroom","parking spaces","floor","animal","furniture"]:
    value=int(input("{} number please:".format(i)))
    value_list.append(value)
value_list
print("the rent is",rf.predict([value_list]),"$")    

As soon as possible I will split data as test and train then I will write all the code again.